#Footy tipping! Part IV

Adding features to improve the logistic/SVM models
+ polynomial -- this is not helping
+ adding features

In [1]:
%matplotlib inline
from IPython.display import Image
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [45]:
# load in training data
data_train = pd.read_csv('afl_train.csv')
# drop if year 2005
data_train = data_train.drop(data_train[data_train.season==2000].index)
data_train.fillna(0)
# make X, y variables
X = data_train[['home_percentage', 'away_percentage', 
                       'home_last_season_percentage', 'away_last_season_percentage']].as_matrix()
y = data_train[['home_team_win']].as_matrix()
m, n = X.shape
X_cols = ['home_percentage', 'away_percentage', 
                       'home_last_season_percentage', 'away_last_season_percentage']
y_col = ['home_team_win']

# load in cross validation data
data_train = pd.read_csv('afl_cval.csv')
X_cv = data_train[['home_percentage', 'away_percentage', 
                       'home_last_season_percentage', 'away_last_season_percentage']].as_matrix()
y_cv = data_train[['home_team_win']].as_matrix()
m_cv, n_cv = X.shape
X_cols_cv = ['home_percentage', 'away_percentage', 
                       'home_last_season_percentage', 'away_last_season_percentage']
y_col_cv = ['home_team_win']

### Making the features polynomial

Restricted so that each feature has the same order polynomial

In [46]:
def poly_features(X_0, p):
    X = np.ones((X_0.shape[0], p))
    X[:,0] = X_0.flatten()
    for j in range(1, p):
        X[:,j] = X[:,0]**(j+1)
    return X

In [105]:
def poly_wrapper(X, p):
    """
    A wrapper to create polynomials for multiple features
    """
    n = X.shape[1]
    df = poly_features(X[:,0], p)
    for i in range(1,n):
        df1 = poly_features(X[:,i], p)
        df = np.concatenate((df, df1), axis=1)
    return df

In [106]:
def feature_normalise(X_0):
    X_p = np.zeros(X_0.shape)
    mu = []
    sigma = []
    for j in range(0, X_0.shape[1]):
        mean_j = np.mean(X_0[:,j])
        std_j = np.std(X_0[:,j])
        X_p[:,j] = (( X_0[:,j] - mean_j) / std_j)
        mu.append(mean_j)
        sigma.append(std_j)
    return X_p, mu, sigma

###Logistic regression

We get a slight increase in performance. No change to bias nor variance

In [92]:
log_reg = LogisticRegression()
pred_log = log_reg.fit(X, y.ravel())
pred_log.score(X, y)

0.66060606060606064

In [109]:
p = 3
X_p, mu, sig = feature_normalise(poly_wrapper(X, p))

In [112]:
log_reg = LogisticRegression()
pred_log = log_reg.fit(X_p, y.ravel())
pred_log.score(X_p, y)

0.66573426573426575

In [117]:
X_cv_p, mu, sig = feature_normalise(poly_wrapper(X_cv, p))
pred_log.score(X_cv_p, y_cv)

0.70707070707070707

###SVM time

With p = 5, the variance is high.
With p = 2, the results are identical.

Feature normalisation helps a little bit.

In [118]:
from sklearn import svm
clf = svm.SVC(kernel='rbf', gamma=1, C=0.5)
clf.fit(X_p, y.ravel())

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=1,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [119]:
clf.score(X_p, y)

0.74405594405594411

In [120]:
clf.score(X_cv_p, y_cv)

0.66666666666666663